In [1]:
cd("/Users/xiaoquer/Library/Mobile Documents/com~apple~CloudDocs/Documents/Work/effective-spinfoam/code/spinfoam-critical-pts")

In [2]:
using LinearAlgebra, Combinatorics, Printf, Dates, PythonCall, Symbolics

sympy = pyimport("sympy")

sympy = pyimport("sympy")

include("PrecisionUtils.jl")
include("SpinAlgebra.jl")
include("SimplexGeometry.jl")
include("TetraNormals.jl")
include("DihedralAngles.jl")
include("LorentzGroup.jl")
include("ThreeDTetra.jl")
include("volume.jl")
include("Su2Su11FromBivector.jl")
include("XiFromSU.jl")
include("FaceNormals3D.jl")
include("KappaFromNormals.jl")
include("GeometryTypes.jl")          # defines GeometryDataset, GeometryCollection
include("GeometryPipeline.jl")
include("GeometryConsistency.jl")
include("KappaOrientation.jl")
include("FourSimplexConnectivity.jl")
include("FaceXiMatching.jl")
include("FaceMatchingChecks.jl")
include("GaugeFixing.jl")
include("CriticalPoints.jl")
include("DefineSymbols.jl")
include("DefineAction.jl")
include("SolveVars.jl")
include("SymbolicToJulia.jl")
include("EOMsHessian.jl")

using .PrecisionUtils: set_big_precision!, parse_numeric_line, set_tolerance!
using .GeometryTypes: GeometryDataset, GeometryCollection
using .GeometryPipeline: run_geometry_pipeline
using .GeometryConsistency: check_sl2c_parallel_transport, check_so13_parallel_transport, check_closure_bivectors
using .KappaOrientation: fix_kappa_signs!
using .FourSimplexConnectivity: build_global_connectivity
using .FaceXiMatching: run_face_xi_matching
using .FaceMatchingChecks: check_all
using .GaugeFixingSU: run_su2_su11_gauge_fix
using .CriticalPoints: compute_bdy_critical_data
using .DefineSymbols: run_define_variables
using .DefineAction: compute_action
using .SolveVars: run_solver
using .SymbolicToJulia: build_action_function, build_gradient_functions, build_hessian_functions, build_argument_vector
using .EOMsHessian: compute_EOMs, compute_Hessian, check_EOMs, evaluate_Hessian

In [3]:
#const ScalarT = Float64
const ScalarT = BigFloat

if ScalarT === BigFloat
    set_big_precision!(256)
    set_tolerance!(1e-20)
else
    set_tolerance!(1e-10)
end

simplices = [
    [1, 2, 3, 4, 5],
    [1, 2, 3, 5, 6]
]

ns = length(simplices)

all_vertices = unique(Iterators.flatten(simplices))
sort!(all_vertices)

Nverts = length(all_vertices)

vertex_coords = Dict{Int, Vector{ScalarT}}()    

coords_lines = [
    "0, 0, 0, 0",
    "0, 0, 0, 1",
    "0, 0, 1, 1",
    "0, 1, 1, 1",
    "1//2, 1, 1, 1",
    "1//2, 0, 1, 1",
]

for (v, line) in zip(all_vertices, coords_lines)
    vertex_coords[v] = parse_numeric_line(line, ScalarT)
end

In [4]:
datasets = GeometryDataset{ScalarT}[]

for (s, simplex) in enumerate(simplices)
    println("\n--- Processing simplex $s with vertices $simplex ---")

    # Build boundary points from user’s vertex coordinates
    bdypoints = [vertex_coords[v] for v in simplex]

    # Run geometry pipeline
    ds = run_geometry_pipeline(bdypoints)
    push!(datasets, ds)
end

geom = GeometryTypes.GeometryCollection(datasets);


--- Processing simplex 1 with vertices [1, 2, 3, 4, 5] ---

--- Processing simplex 2 with vertices [1, 2, 3, 5, 6] ---


In [5]:
for (idx, simplex) in enumerate(geom.simplex)
    check_sl2c_parallel_transport(simplex.solgsl2c, simplex.bdybivec55)
    check_so13_parallel_transport(simplex.solgso13, simplex.bdybivec4d55)
    check_closure_bivectors(simplex.kappa, simplex.areas, simplex.bdybivec55)
end

✓ SL(2,C) parallel transport satisfied (max residual = 1.330922959325613670785575500700333790721517116199708682358791833377792063318259e-77).
✓ SO(1,3) parallel transport satisfied (max residual = 9.229015197838843517857585188446592577841084396073233251581187104279989919875818e-78)
✓ Closure condition satisfied for bivectors (max residual = 8.636168555094444625386351862800399571116000364436281385023703470168591803162427e-78)
✓ SL(2,C) parallel transport satisfied (max residual = 2.335361141170707923071153214827985271452361999751143049761442414371791687176604e-77).
✓ SO(1,3) parallel transport satisfied (max residual = 7.961737133231731631245126357720371009783450811487682073015399407890654258760537e-78)
✓ Closure condition satisfied for bivectors (max residual = 8.636168555094444625386351862800399571116000364436281385023703470168591803162427e-78)


In [6]:
fix_kappa_signs!(simplices, geom);

[1, 2]


In [7]:
conn = build_global_connectivity(simplices, geom)
push!(geom.connectivity, conn);

In [8]:
run_face_xi_matching(geom);

  SL(2,C) matrices updated:      ✓
  Boundary bivectors updated:    ✓
  boundary ξ variables updated:  ✓
  SU(2)/SU(1,1) elements updated: ✓
  SO(1,3) frames corrected:       ✓



In [9]:
check_all(geom)


   FACE–XI GEOMETRY CHECKS

✓ Face matching satisfied (max residual = 6.304479601281264453947185039589470456952015198386299840182050802150653872795539e-78).
✓ SL(2,C) parallel transport satisfied (max residual = 2.425790183822456346047297812942231600946620449573778512699429527964649613456347e-77).
✓ Closure condition satisfied (max residual = 1.026709329865829110265857575281794725658673120559649876478322566380038108804927e-77).

------------------------------
✓ All geometric consistency checks PASSED.
------------------------------



In [10]:
run_su2_su11_gauge_fix(geom);

In [11]:
run_define_variables(geom);

In [12]:
sd, γsym = run_solver(geom)

S = compute_action(geom)
dS = compute_EOMs(S, sd)
H  = compute_Hessian(S, sd);

In [13]:
@variables γ

S_fn, labels = build_action_function(S, sd);

grad_fns = build_gradient_functions(dS, sd)
hess_fns = build_hessian_functions(H, sd);

In [14]:
args = build_argument_vector(sd, labels, γ);
S_sym = simplify(S_fn(args...))

(im*(-0.6670596978199762053462312037003038558542290891072528300183556953884316032617968 + 0.9367506454560388817679426726559190624819351551905883204891046230231583517707618γ)) / γ

In [22]:
include("EOMsHessian.jl")
using .EOMsHessian

In [17]:
EOMsHessian.check_EOMs(grad_fns, sd)

✔ All equations of motion satisfied (γ = 1, tol = 1.0e-20).


In [23]:
Hess, _ = EOMsHessian.evaluate_hessian(hess_fns, sd; γ = one(ScalarT));

In [24]:
det(Hess)

-1.10714372579382856388585459244545082157633705792981574642633101658774027828295e-09 + 2.707704778746195938340000312597498525082827401302042552867242434312313668393776e-11im